'''
Basic Multi GPU computation example using TensorFlow library.
'''

'''
This tutorial requires your machine to have 2 GPUs

"/cpu:0": The CPU of your machine.

"/gpu:0": The first GPU of your machine

"/gpu:1": The second GPU of your machine
'''

In [ ]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
import datetime

In [ ]:
# Processing Units logs
log_device_placement = True

# Num of multiplications to perform
n = 10




Example: compute A^n + B^n on 2 GPUs


In [ ]:
# Create random large matrix
A = np.random.rand(1000, 1000).astype('float32')
B = np.random.rand(1000, 1000).astype('float32')

# Create a graph to store results
c0 = []
c1 = []
c2 = []

In [ ]:
def matpow(M, n):
    if n < 1: #Abstract cases where n < 1
        return M
    else:
        return tf.matmul(M, matpow(M, n-1))


'''
Only CPU computing
'''

In [ ]:
with tf.device('/cpu:0'):
    a = tf.placeholder(tf.float32, [1000, 1000])
    b = tf.placeholder(tf.float32, [1000, 1000])
    # Compute A^n and B^n and store results in c0
    c0.append(matpow(a, n))
    c0.append(matpow(b, n))
    sum = tf.add_n(c0) #Addition of all elements in c0, i.e. A^n + B^n

t1_0 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    # Run the op.
    sess.run(sum, {a:A, b:B})
t2_0 = datetime.datetime.now()

'''
Single GPU computing
'''

In [ ]:
with tf.device('/gpu:0'):
    a = tf.placeholder(tf.float32, [1000, 1000])
    b = tf.placeholder(tf.float32, [1000, 1000])
    # Compute A^n and B^n and store results in c1
    c1.append(matpow(a, n))
    c1.append(matpow(b, n))

with tf.device('/cpu:0'):
    sum = tf.add_n(c1) #Addition of all elements in c1, i.e. A^n + B^n

t1_1 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    # Run the op.
    sess.run(sum, {a:A, b:B})
t2_1 = datetime.datetime.now()


'''
Multi GPU computing
'''

In [ ]:
# GPU:0 computes A^n
with tf.device('/gpu:0'):
    # Compute A^n and store result in c2
    a = tf.placeholder(tf.float32, [1000, 1000])
    c2.append(matpow(a, n))

# GPU:1 computes B^n
with tf.device('/gpu:1'):
    # Compute B^n and store result in c2
    b = tf.placeholder(tf.float32, [1000, 1000])
    c2.append(matpow(b, n))

with tf.device('/cpu:0'):
    sum = tf.add_n(c2) #Addition of all elements in c2, i.e. A^n + B^n

t1_2 = datetime.datetime.now()
with tf.Session(config=tf.ConfigProto(log_device_placement=log_device_placement)) as sess:
    # Run the op.
    sess.run(sum, {a:A, b:B})
t2_2 = datetime.datetime.now()

In [ ]:
print("Only CPU computation time: " + str(t2_0-t1_0))
print("Single GPU computation time: " + str(t2_1-t1_1))
# print("Multi GPU computation time: " + str(t2_2-t1_2))

Results on 8 cores with 2 GTX-980 with dimensions: [10000,10000]
 * Single GPU computation time: 0:00:11.277449
 * Multi GPU computation time: 0:00:07.131701